# Assignment 7

## Deadline

Wednesday, November 19, 23:59.

### Task

Write a Python function `analyze_flight_tickets(tickets)` that accepts a list of flight tickets as input. Each ticket is represented as a dictionary with the following keys:  

- `passenger` - customer name; type `str`  
- `origin` - airport code of departure; type `str`  
- `destination` - airport code of destination; type `str`  
- `price` - flight price in CZK; type `float` or `int`  
- `duration` - flight duration in minutes; type `float` or `int`  
- `distance` - distance traveled in km; type `float` or `int`  

Assume that all provided tickets contain all required fields with non-empty values.  

#### Goal

The goal of the function is to analyze the tickets and return the following data as a **dictionary** with the following keys and values:  

1. **`destination_stats`** - Destination popularity statistics. **Dictionary** where:  
   - Keys are destination airport codes (`str`)  
   - Values are the number of flights to that destination (`int`)  

2. **`passenger_stats`** - Passenger statistics. **Dictionary** where:  
   - Keys are passenger names  
   - Values are dictionaries containing:  
     - `total_duration` - Cumulative flight time for the passenger  
     - `total_distance` - Cumulative distance traveled by the passenger  
     - `total_price` - Cumulative ticket price for the passenger  
     - `visited_airports` - Set of airports visited by the passenger (includes both departure and arrival airports)  

3. **`total_revenue`** - Total revenue from ticket sales (numeric value).  

#### Input data conditions

- `price`: Positive number (`int` or `float`).  
- `distance` and `duration`: Positive numbers (`int` or `float`) convertible to integers without rounding error (e.g., `1` or `1.0` are valid; `1.1` is invalid).
- All fields `price`, `distance`, and `duration` must be positive (i.e., `price > 0`, `distance > 0`, `duration > 0`).  
- Route validity check: If `origin` equals `destination`, the flight is invalid (values must differ).  

If any condition fails, the function returns `None`.  

### Implementation guidelines 

- Do not use external libraries (only standard Python).
- Do not use global variables.
- Do not modify the original data.
- Do not use `input()` in the final solution (input data is provided automatically).  
- Do not use `print()` in the final solution.  
- Create small helper functions as needed.
- For sample input/output data, refer to the public test cases section.

In [ ]:
#

#! Helper Functions
#* Positive int/float check
def isPositiveNumber(suspect):
     if suspect > 0 and isinstance(suspect, (int, float)) and not isinstance(suspect, bool):
          return True
     return False

#* Rounding check
def canBeRounded(suspect):
     if suspect == int(suspect):
          return True
     return False

#* Destination existance
def doesThisDestinationAlreadyExist(destination, dictionary):
     if destination in dictionary["destination_stats"]:
          return True
     return False

#* Person existance
def doesThisPersonAlreadyExist(person, dictionary):
     if person in dictionary["passenger_stats"]:
          return True
     return False

#! Analyze Flight Tickets
def analyze_flight_tickets(tickets):
     """
     The main function that analyzes flight tickets
     :param tickets: list of tickets (dictionaries)
     :return: dictionary with destination_stats, passenger_stats and total_revenue keys
     """

     #* Variables
     resultObject = {
          "destination_stats": {},
          "passenger_stats": {},
          "total_revenue": 0
     }
     revenue = 0

     #* Forloop
     for ticket in tickets:
          # variables
          passenger = ticket["passenger"]
          origin = ticket["origin"]
          destination = ticket["destination"]
          price = ticket["price"]
          duration = ticket["duration"]
          distance = ticket["distance"]

          # input conditions
          if not isPositiveNumber(price) or not isPositiveNumber(duration) or not isPositiveNumber(distance):
               return None
          if not canBeRounded(duration) or not canBeRounded(distance):
               return None
          if origin == destination:
               return None
          
          # fix data types
          duration = int(duration)
          distance = int(distance)

          # destination stats
          if doesThisDestinationAlreadyExist(destination, resultObject):
               resultObject["destination_stats"][destination] += 1
          else:
               resultObject["destination_stats"][destination] = 1

          # passenger stats
          if doesThisPersonAlreadyExist(passenger, resultObject):
               resultObject["passenger_stats"][passenger]["total_duration"] += duration
               resultObject["passenger_stats"][passenger]["total_distance"] += distance
               resultObject["passenger_stats"][passenger]["total_price"] += price
               resultObject["passenger_stats"][passenger]["visited_airports"].add(origin)
               resultObject["passenger_stats"][passenger]["visited_airports"].add(destination)
          else:
               resultObject["passenger_stats"][passenger] = {
                    "total_duration": duration,
                    "total_distance": distance,
                    "total_price": price,
                    "visited_airports": {origin, destination}
               }

          # revenue
          revenue += price

     #* Output
     resultObject["total_revenue"] = revenue
     return resultObject

✓ Test 1 passed: Basic functionality
✓ Test 2 passed: Same passenger, multiple flights
✓ Test 3 passed: Float to int conversion
✓ Test 4 passed: Negative price rejected
✓ Test 5 passed: Zero duration rejected
✓ Test 6 passed: Same origin/destination rejected
✓ Test 7 passed: Non-roundable float rejected
✓ Test 8 passed: Empty list handled
✓ Test 9 passed: Invalid data after valid rejected
✓ Test 10 passed: Multiple passengers, same route
✓ Test 11 passed: Very small positive values accepted
✓ Test 12 passed: Round trip handled correctly

🎉 All tests passed!


The following cells contain public tests that you can use for basic validation of your solution. **Click on the validate button before submitting!**

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
from pprint import pprint


def validate_basic_properties(result, solution):
    if solution is None:
        assert result is None, "The return value should be None"
        return
    
    assert isinstance(result, dict), "The returned value is not a dictionary"
    assert set(result.keys()) == {'destination_stats', 'passenger_stats',
                                  'total_revenue'}, "The keys of the returned dictionary do not match the expected keys"
    assert isinstance(result['total_revenue'], int) or isinstance(result['total_revenue'],
                                                                  float), 'total_revenue is not a number (int or float)'
    assert isinstance(result['destination_stats'], dict), "destination_stats is not a dictionary"
    assert isinstance(result['passenger_stats'], dict), "passenger_stats is not a dictionary"

    for airport, frequency in result['destination_stats'].items():
        assert isinstance(airport, str), "Airport name (code) is not a string"
        assert len(airport) > 0, "Airport name (code) is an empty string"
        assert isinstance(frequency, int), "Frequency is not an integer"
        
    for passenger, stats in result['passenger_stats'].items():
        assert isinstance(passenger, str), "Passenger name in the `passenger_stats` dictionary is not a string"
        assert isinstance(stats, dict), f"Statistics for passenger {passenger} in the `passenger_stats` dictionary are not a dictionary"
        assert set(stats.keys()) == {'total_duration', 'total_distance', 'total_price', 'visited_airports'}, \
            f"The keys of the statistics dictionary for passenger {passenger} do not match the expected keys"
        for key in ('total_duration', 'total_distance', 'total_price'):
            assert isinstance(stats[key], int) or isinstance(stats[key], float), \
                f"The value of {key} for passenger {passenger} in the `passenger_stats` dictionary is not a number (int or float)"
        assert isinstance(stats['visited_airports'], set), \
            f"The value of `visited_airports` for passenger {passenger} in the `passenger_stats` dictionary is not a set"
        assert all(isinstance(item, str) for item in stats['visited_airports']), \
            f"The value of `visited_airports` for passenger {passenger} in the `passenger_stats` dictionary contains an item that is not a string"


def check_solution_numbers(testcase):
    result = analyze_flight_tickets(testcase['test'])
    # validate_basic_properties(result, testcase['solution'])
    try:
        validate_basic_properties(result, testcase['solution'])
        if testcase['solution'] is None:
            print('Test OK')
            return
        # allow for rounding error on floats
        assert abs(result['total_revenue'] - testcase['solution']['total_revenue']) <= 1, \
                "The value of `total_revenue` is not correct"
        
        assert set(result['passenger_stats'].keys()) == set(testcase['solution']['passenger_stats'].keys()), \
            "The keys of the `passenger_stats` dictionary do not match the expected keys"
        
        for passenger, stats in result['passenger_stats'].items():
            assert abs(stats['total_price'] - testcase['solution']['passenger_stats'][passenger]['total_price']) <= 1, \
                f"The value of `total_price` is not correct for passenger {passenger}"
            assert abs(stats['total_distance'] - testcase['solution']['passenger_stats'][passenger]['total_distance']) <= 1, \
                f"The value of `total_distance` is not correct for passenger {passenger}"
            assert abs(stats['total_duration'] - testcase['solution']['passenger_stats'][passenger]['total_duration']) <= 1, \
                f"The value of `total_duration` is not correct for passenger {passenger}"
        
        assert abs(result['total_revenue'] - sum(s['total_price'] for s in result['passenger_stats'].values())) <= 1, \
            "The sum of `total_price` for all passengers does not match the value of `total_revenue`"
        
    except AssertionError:
        print('Test Failed')
        print('Test case:')
        pprint(testcase['test'])
        print('Your solution:')
        pprint(result)
        print('Correct solution:')
        pprint(testcase['solution'])
        raise
    print('Test OK')

def check_solution_airports(testcase):
    result = analyze_flight_tickets(testcase['test'])

    try:
        validate_basic_properties(result, testcase['solution'])
        if testcase['solution'] is None:
            print('Test OK')
            return
        assert result['destination_stats'] == testcase['solution']['destination_stats'], \
            "The value of `destination_stats` is not correct"
        assert sum(result['destination_stats'].values()) == len(testcase['test']), \
            "The sum of the popularity values in `destination_stats` does not match the total number of flights"
        for passenger, stats in result['passenger_stats'].items():
            assert stats['visited_airports'] == testcase['solution']['passenger_stats'][passenger]['visited_airports'], \
                f"The value of `visited_airports` is not correct for passenger {passenger}"
            
    except AssertionError:
        print('Test Failed')
        print('Test case:')
        pprint(testcase['test'])
        print('Your solution:')
        pprint(result)
        print('Correct solution:')
        pprint(testcase['solution'])
        raise
    print('Test OK')

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!

# definitions of public test cases

t_empty = {
    'test': [],
    'solution': {'destination_stats': {}, 'passenger_stats': {}, 'total_revenue': 0}
}

t_single = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940}
    ],
    'solution': {'destination_stats': {'GVE': 1}, 'passenger_stats': {'John': {'total_duration': 73, 'total_distance': 940, 'total_price': 3500, 'visited_airports': {'GVE', 'PRG'}}}, 'total_revenue': 3500}
}

t_multi_user = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940},
        {'passenger': 'Anna', 'origin': 'PRG', 'destination': 'ALY', 'price': 8000, 'duration': 195, 'distance': 3002},
        {'passenger': 'Peter', 'origin': 'LAX', 'destination': 'LBP', 'price': 23000, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 1, 'ALY': 1, 'LBP': 1}, 'passenger_stats': {'John': {'total_duration': 73, 'total_distance': 940, 'total_price': 3500, 'visited_airports': {'PRG', 'GVE'}}, 'Anna': {'total_duration': 195, 'total_distance': 3002, 'total_price': 8000, 'visited_airports': {'ALY', 'PRG'}}, 'Peter': {'total_duration': 730, 'total_distance': 13012, 'total_price': 23000, 'visited_airports': {'LAX', 'LBP'}}}, 'total_revenue': 34500}
}

t_multi_flight_single_user = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940},
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'MYR', 'price': 8000, 'duration': 195, 'distance': 3002},
        {'passenger': 'John', 'origin': 'LAX', 'destination': 'GVE', 'price': 23000, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 2, 'MYR': 1}, 'passenger_stats': {'John': {'total_duration': 998, 'total_distance': 16954, 'total_price': 34500, 'visited_airports': {'GVE', 'PRG', 'MYR', 'LAX'}}}, 'total_revenue': 34500}
}

t_multi_flight_multi_user = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940},
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'MYR', 'price': 8000, 'duration': 195, 'distance': 3002},
        {'passenger': 'John', 'origin': 'LAX', 'destination': 'GVE', 'price': 23000, 'duration': 730, 'distance': 13012},
        {'passenger': 'Anna', 'origin': 'PRG', 'destination': 'GVE', 'price': 2462, 'duration': 375, 'distance': 4621},
        {'passenger': 'Anna', 'origin': 'ABC', 'destination': 'ALY', 'price': 1379, 'duration': 132, 'distance': 672},
        {'passenger': 'Anna', 'origin': 'CDE', 'destination': 'ORB', 'price': 738, 'duration': 110, 'distance': 10555},
        {'passenger': 'Peter', 'origin': 'LAX', 'destination': 'LBP', 'price': 23000, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 3, 'MYR': 1, 'ALY': 1, 'ORB': 1, 'LBP': 1}, 'passenger_stats': {'John': {'total_duration': 998, 'total_distance': 16954, 'total_price': 34500, 'visited_airports': {'MYR', 'GVE', 'PRG', 'LAX'}}, 'Anna': {'total_duration': 617, 'total_distance': 15848, 'total_price': 4579, 'visited_airports': {'GVE', 'PRG', 'ORB', 'ABC', 'ALY', 'CDE'}}, 'Peter': {'total_duration': 730, 'total_distance': 13012, 'total_price': 23000, 'visited_airports': {'LBP', 'LAX'}}}, 'total_revenue': 62079}
}

t_float_price = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500.6, 'duration': 73, 'distance': 940},
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'MYR', 'price': 8000.1, 'duration': 195, 'distance': 3002},
        {'passenger': 'John', 'origin': 'LAX', 'destination': 'GVE', 'price': 23000.5, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 2, 'MYR': 1}, 'passenger_stats': {'John': {'total_duration': 998, 'total_distance': 16954, 'total_price': 34501.2, 'visited_airports': {'GVE', 'PRG', 'MYR', 'LAX'}}}, 'total_revenue': 34501.2}
}

t_float_duration = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73.0, 'distance': 940},
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'MYR', 'price': 8000, 'duration': 195.0, 'distance': 3002},
        {'passenger': 'John', 'origin': 'LAX', 'destination': 'GVE', 'price': 23000, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 2, 'MYR': 1}, 'passenger_stats': {'John': {'total_duration': 998.0, 'total_distance': 16954, 'total_price': 34500, 'visited_airports': {'LAX', 'PRG', 'MYR', 'GVE'}}}, 'total_revenue': 34500}
}

t_float_distance = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940.0},
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'MYR', 'price': 8000, 'duration': 195, 'distance': 3002.0},
        {'passenger': 'John', 'origin': 'LAX', 'destination': 'GVE', 'price': 23000, 'duration': 730, 'distance': 13012}
    ],
    'solution': {'destination_stats': {'GVE': 2, 'MYR': 1}, 'passenger_stats': {'John': {'total_duration': 998, 'total_distance': 16954.0, 'total_price': 34500, 'visited_airports': {'LAX', 'PRG', 'MYR', 'GVE'}}}, 'total_revenue': 34500}
}


t_none_airport = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'PRG', 'price': 3500, 'duration': 73, 'distance': 940}
    ],
    'solution': None
}

t_none_duration_float = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73.5, 'distance': 940}
    ],
    'solution': None
}

t_none_duration_negative = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': -73, 'distance': 940}
    ],
    'solution': None
}

t_none_duration_zero = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 0, 'distance': 940}
    ],
    'solution': None
}


t_none_distance_float = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 940.5}
    ],
    'solution': None
}

t_none_distance_negative = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': -940}
    ],
    'solution': None
}

t_none_distance_zero = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 3500, 'duration': 73, 'distance': 0}
    ],
    'solution': None
}

t_none_price_negative = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': -3500, 'duration': 73, 'distance': 940}
    ],
    'solution': None
}

t_none_price_zero = {
    'test': [
        {'passenger': 'John', 'origin': 'PRG', 'destination': 'GVE', 'price': 0, 'duration': 73, 'distance': 940}
    ],
    'solution': None
}


In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_numbers(t_empty)
check_solution_numbers(t_single)
check_solution_numbers(t_multi_user)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_airports(t_empty)
check_solution_airports(t_single)
check_solution_airports(t_multi_user)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_numbers(t_multi_flight_single_user)
check_solution_numbers(t_multi_flight_multi_user)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_airports(t_multi_flight_single_user)
check_solution_airports(t_multi_flight_multi_user)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_numbers(t_float_price)
check_solution_numbers(t_float_duration)
check_solution_numbers(t_float_distance)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_airports(t_float_price)
check_solution_airports(t_float_duration)
check_solution_airports(t_float_distance)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_numbers(t_none_airport)
check_solution_numbers(t_none_duration_float)
check_solution_numbers(t_none_duration_negative)
check_solution_numbers(t_none_duration_zero)
check_solution_numbers(t_none_distance_float)
check_solution_numbers(t_none_distance_negative)
check_solution_numbers(t_none_distance_zero)
check_solution_numbers(t_none_price_negative)
check_solution_numbers(t_none_price_zero)

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!
check_solution_airports(t_none_airport)
check_solution_airports(t_none_duration_float)
check_solution_airports(t_none_duration_negative)
check_solution_airports(t_none_duration_zero)
check_solution_airports(t_none_distance_float)
check_solution_airports(t_none_distance_negative)
check_solution_airports(t_none_distance_zero)
check_solution_airports(t_none_price_negative)
check_solution_airports(t_none_price_zero)

The following cells contain hidden tests that are evaluated during automatic grading after your submission. The tests check that your functions are sufficiently general.

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!

In [ ]:
# This is a read-only cell used for automatic validation - do not edit, delete, or move!